In [2]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import pandas as pd
import os
import re
import csv
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from datasets import load_dataset
from transformers import AutoTokenizer,create_optimizer,TFAutoModel

In [34]:
BATCH_SIZE = 128
MAX_LENGTH = 64

## Dataset Preparation

In [4]:
filepath_train = 'train-v0.3.csv'
filepath_catalogue = 'product_catalogue-v0.3.csv'

In [5]:
df_catalogue = pd.read_csv(filepath_catalogue)
df_train = pd.read_csv(filepath_train)

In [6]:
df_train.head()

,query_id,query,query_locale,product_id,esci_label
0,0,# 2 pencils not sharpened,us,B0000AQO0O,exact
1,0,# 2 pencils not sharpened,us,B0002LCZV4,exact
2,0,# 2 pencils not sharpened,us,B00125Q75Y,exact
3,0,# 2 pencils not sharpened,us,B001AZ1D3C,exact
4,0,# 2 pencils not sharpened,us,B001B097KC,exact


In [8]:
df_train.loc[df_train['query_locale']=='jp']

,query_id,query,query_locale,product_id,esci_label
572650,26004,0係,jp,B00FNJ1CTQ,substitute
572651,26004,0係,jp,B00FW60P84,irrelevant
572652,26004,0係,jp,B00GAYCNDM,substitute
572653,26004,0係,jp,B00MFBKQKG,substitute
572654,26004,0係,jp,B01G1YXU28,irrelevant
...,...,...,...,...,...
781739,33803,針なしほっちきす,jp,B08XGQ9RH7,substitute
781740,33803,針なしほっちきす,jp,B0987RGRF2,exact
781741,33803,針なしほっちきす,jp,B099NFJWP6,exact
781742,33803,針なしほっちきす,jp,B09F3B413J,exact


In [9]:
df_catalogue

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpene...",NaN,144 woodcase #2 HB pencils made from high-qual...,Amazon Basics,Yellow,us
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser,...",<p><strong>BACK TO BAZIC</strong></p><p>Our go...,&#11088; UN-SHARPENED #2 PREMIUM PENCILS. Each...,BAZIC Products,12-count,us
2,B07G7F6JZ6,Emraw Pre Sharpened Round Primary Size No 2 Ju...,<p><b>Emraw Pre-Sharpened #2 HB Wood Pencils -...,✓ PACK OF 8 NUMBER 2 PRESHARPENED BEGINNERS PE...,Emraw,Yellow,us
3,B07JZJLHCF,Emraw Pre Sharpened Triangular Primary Size No...,<p><b>Emraw Pre-Sharpened #2 HB Wood Pencils -...,✓ PACK OF 6 NUMBER 2 PRESHARPENED BEGINNERS PE...,Emraw,Yellow,us
4,B07MGKC3DD,"BIC Evolution Cased Pencil, #2 Lead, Gray Barr...",NaN,Premium #2 HB lead pencils with break-resistan...,Design House,Gray,us
...,...,...,...,...,...,...,...
883863,B094NN6T8V,アイリスプラザ コーナーソファ 3点セット 高反発 2人掛け 3人掛け ブラウン 394310,NaN,商品サイズ（cm）：幅約154×奥行約109×高さ約33（※組み合わせ時）\n商品重量：約6...,アイリスプラザ(IRIS PLAZA),ブラウン,jp
883864,B096RD95K9,Uping テーブル・ベッドの高さ調節が簡単にできる ベッドの高さをあげる足 4個セット 高...,B096RD95K9,サイズ:約16.5×11.7×8.2cm\n総重量:約1.23kg\n材質:ABSプラスチッ...,Uping,NaN,jp
883865,B099Z4HXH1,アイリスプラザ ソファ ソファー ローソファ ローソファー 1人掛け 1人用 幅約108cm...,NaN,商品サイズ(cm):幅約108×奥行約44×高さ約62\n商品重量:約15kg\n材質 表面...,アイリスプラザ(IRIS PLAZA),グレー,jp
883866,B09B9QMMSF,足入れヒーター 足温器 電気足温器 1～6時間タイマー 丸洗い可能 省エネ あったか脚入れヒ...,NaN,【腰からスッポリ包む、一人様用のごたつ】 つま先・腰・お腹まですっぽり包む寝袋のようなデザイ...,タサイスク,ブラウン,jp


In [10]:
def get_product_title(product_id):
    options = []
    options.append(str(product_id))
    matched_row = df_catalogue.loc[df_catalogue['product_id'].isin(options)]
    return matched_row['product_title'][int(str(matched_row['product_title']).split("   ")[0])]

In [11]:
get_product_title("B08N6NCKRX")

'面部保护 防雾 耐用 5个 透明面罩 健康盾牌 塑料面巾 完全可见 透气 可重复使用 成人 口罩 方便 打招呼 防护 (Clear)'

In [12]:
df_catalogue.shape

(883868, 7)

In [13]:
df_train.shape

(781744, 5)

## Dataset Generation

In [14]:
df_train['product'] = df_train['product_id'].apply(get_product_title)

df_to_write = df_train[['query', 'product', 'esci_label']]


output_file = 'csv_file.csv'
df_to_write.to_csv(output_file, index=False, encoding='UTF-8')


In [16]:
filepath = 'csv_file.csv'
train_data = pd.read_csv(filepath)

In [17]:
train_data

,query,product,esci_label
0,# 2 pencils not sharpened,"Ticonderoga Beginner Pencils, Wood-Cased #2 HB...",exact
1,# 2 pencils not sharpened,"TICONDEROGA Tri-Write Triangular Pencils, Stan...",exact
2,# 2 pencils not sharpened,"TICONDEROGA Pencils, Wood-Cased, Unsharpened, ...",exact
3,# 2 pencils not sharpened,"Ticonderoga Pencils, Wood-Cased Graphite #2 HB...",exact
4,# 2 pencils not sharpened,"Ticonderoga Laddie Pencils, Wood-Cased #2 HB S...",exact
...,...,...,...
781739,針なしほっちきす,マックス ホッチキス 20枚とじ HD-10GK/BJ ベージュ,substitute
781740,針なしほっちきす,ホッチキス 360°回転 幅広いアプリケーション ​調整可能 ステープラー 縦/横 オフィス...,exact
781741,針なしほっちきす,マックス ホッチキス バイモ11 ロング 中とじ 100本装填 ホワイト HD-11DB/W,exact
781742,針なしほっちきす,ホチキス 卓上 ステープラー 半透明 アクリル 丈夫 おしゃれ 業務用ホッチキス 24/6号...,exact


## Data Preprocessing

In [18]:
dataset = load_dataset('csv', data_files=filepath)

Generating train split: 0 examples [00:00, ? examples/s]

In [19]:
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'product', 'esci_label'],
        num_rows: 781744
    })
})

In [29]:
dataset['train'][0]

{'query': '# 2 pencils not sharpened',
 'product': 'Ticonderoga Beginner Pencils, Wood-Cased #2 HB Soft, With Eraser, Yellow, 12-Pack (13308)',
 'esci_label': 'exact'}

In [20]:
def get_label(label):
    if label=='exact':
        return 1.0
    elif label=='substitute':
        return 0.7
    elif label=='complement':
        return 0.5
    else:
        return 0

In [21]:
model_id = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

c:\Users\risha\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\risha\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [30]:
def preprocess(dataset):
  if dataset['product']==None:
    dataset['product']=dataset['query']

  dataset['input_ids_query']=[]
  dataset['token_type_ids_query']=[]
  dataset['attention_mask_query']=[]

  dataset['input_ids_product']=[]
  dataset['token_type_ids_product']=[]
  dataset['attention_mask_product']=[]
  
  tokenized_output_query=tokenizer(dataset['query'],max_length=MAX_LENGTH,padding='max_length',truncation=True)
  tokenized_output_product=tokenizer(dataset['product'],max_length=MAX_LENGTH,padding='max_length',truncation=True)
  
  dataset['input_ids_query'].append(tokenized_output_query['input_ids'])
  dataset['token_type_ids_query'].append(tokenized_output_query['token_type_ids'])
  dataset['attention_mask_query'].append(tokenized_output_query['attention_mask'])
  
  dataset['input_ids_product'].append(tokenized_output_product['input_ids'])
  dataset['token_type_ids_product'].append(tokenized_output_product['token_type_ids'])
  dataset['attention_mask_product'].append(tokenized_output_product['attention_mask'])

  dataset['esci_label']=get_label(dataset['esci_label'])
  
  return dataset

In [31]:
prep_dataset = dataset.map(preprocess)

Map:   0%|          | 0/781744 [00:00<?, ? examples/s]

In [32]:
prep_dataset['train'][22:23]

{'query': ['# 2 pencils not sharpened'],
 'product': ['Maped Essentials Triangular Graphite #2 Pencils, Pack of 12 (851779ZT)'],
 'esci_label': [1.0],
 'input_ids_query': [[[0,
    468,
    116,
    5551,
    13003,
    7,
    959,
    189173,
    33,
    297,
    2,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1]]],
 'token_type_ids_query': [[[0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0

In [36]:
tf_dataset = prep_dataset['train'].to_tf_dataset(
    columns=['input_ids_query', 'token_type_ids_query', 'attention_mask_query', 'input_ids_product', 'token_type_ids_product', 'attention_mask_product', 'esci_label'],
    shuffle=True,
    batch_size=BATCH_SIZE
)

In [37]:
tf_dataset

<_PrefetchDataset element_spec={'esci_label': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'input_ids_query': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'token_type_ids_query': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'attention_mask_query': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'input_ids_product': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'token_type_ids_product': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'attention_mask_product': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None)}>

In [38]:
for i in tf_dataset.take(1):
    print(i)

{'esci_label': <tf.Tensor: shape=(128,), dtype=float32, numpy=
array([1. , 0.7, 1. , 0.5, 1. , 1. , 0.7, 1. , 0.7, 0.7, 1. , 0. , 0.7,
       0.7, 1. , 0. , 0.7, 1. , 1. , 0.5, 0.7, 0.7, 1. , 1. , 0.7, 1. ,
       1. , 0.7, 1. , 0.7, 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       0. , 1. , 1. , 0. , 0.7, 0. , 1. , 1. , 1. , 0.7, 0. , 0.7, 1. ,
       1. , 0.7, 0.7, 1. , 1. , 0.7, 1. , 1. , 0.7, 0.7, 0. , 1. , 1. ,
       0.7, 0. , 0.7, 1. , 0.7, 0.7, 0.7, 0. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 0.7, 0.7, 0.7, 0.7, 0. , 0.7, 1. , 0.7, 1. , 0.7, 1. ,
       1. , 0.7, 0. , 0.7, 0.7, 0. , 0.7, 0.5, 0.7, 1. , 1. , 0.7, 0.7,
       0. , 1. , 1. , 0.5, 1. , 1. , 0.7, 1. , 0.5, 0.7, 1. , 1. , 0.7,
       1. , 1. , 0.7, 0.7, 1. , 1. , 0.5, 1. , 1. , 1. , 1. ],
      dtype=float32)>, 'input_ids_query': <tf.Tensor: shape=(128, 1, 64), dtype=int64, numpy=
array([[[     0,      6, 109348, ...,      1,      1,      1]],

       [[     0,      6,  11264, ...,      1,      1,      1]],

  

## Modeling

In [39]:
model = TFAutoModel.from_pretrained(model_id)
model.summary()

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  117653760 
                                                                 
Total params: 117653760 (448.81 MB)
Trainable params: 117653760 (448.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [47]:
# class SentenceTransformer(tf.keras.Model):
#     def __init__(self, model):
#         super(SentenceTransformer, self).__init__()
#         self.model = model
#         self.dense = Dense(1, activation='sigmoid')

#     def compile(self, optimizer, loss_fn):
#         super(SentenceTransformer, self).compile()
#         self.optimizer = optimizer
#         self.loss_fn = loss_fn
#         self.loss_metrics = tf.keras.metrics.Mean(name='loss')

#     @property
#     def metrics(self):
#         return [self.loss_metrics]
    
#     def mean_pooling(self, model_ouput, attention_mask):
#         token_embeddings = model_ouput[0]

#         input_mask_expanded = tf.cast(
#             tf.broadcast_to(tf.expand_dims(attention_mask, -1), tf.shape(token_embeddings)),
#             tf.float32
#         )
#         return tf.math.reduce_sum(token_embeddings*input_mask_expanded, axis=1)/tf.clip_by_value(tf.math.reduce_sum(input_mask_expanded, axis=1), 1e-9, tf.float32.max)
    
#     def train_steps(self, train_data):

#         query = {'input_ids':train_data['input_ids_query'][:,0,:],
#                  'token_type_ids':train_data['token_type_ids_query'][:,0,:],
#                  'attention_mask':train_data['attention_mask_query'][:,0,:]}
        
#         product = {'input_ids':train_data['input_ids_product'][:,0,:],
#                    'token_type_ids':train_data['token_type_ids_product'][:,0,:],
#                    'attention_mask':train_data['attention_mask_product'][:,0,:]}
        
#         labels = train_data['esci_label']

#         with tf.GradientTape() as recorder:
#             query_predictions = self.model(query)
#             pred_query = self.mean_pooling(query_predictions, train_data['attention_mask_query'][:,0,:])

#             product_predictions = self.model(product)
#             pred_product = self.mean_pooling(product_predictions, train_data['attention_mask_product'][:,0,:])

#             pred_concat = tf.concat([pred_query, pred_product, tf.abs(pred_query - pred_product)], axis=-1)

#             predictions = self.dense(pred_concat)
#             loss = self.loss_fn(labels, predictions)

#         partial_derivatives = recorder.gradient(loss, self.model.trainable_weights)
#         self.optimizer.apply_gradients(zip(partial_derivatives, self.model.trainable_weights))

#         self.loss_metrics.update_state(loss)

#         return {'loss':self.loss_metrics.result(),}

In [56]:
class SentenceTransformer(tf.keras.Model):
    def __init__(self, model):
        super(SentenceTransformer, self).__init__()
        self.model = model
        self.dense = Dense(1, activation='sigmoid')

    def compile(self, optimizer, loss_fn):
        super(SentenceTransformer, self).compile()
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.loss_metrics = tf.keras.metrics.Mean(name='loss')

    @property
    def metrics(self):
        return [self.loss_metrics]
    
    def mean_pooling(self, model_output, attention_mask):
        """
        Applies mean pooling to the token embeddings using the attention mask.
        """
        token_embeddings = model_output[0]  # First output contains token embeddings

        # Expand the attention mask for broadcasting
        input_mask_expanded = tf.cast(
            tf.broadcast_to(tf.expand_dims(attention_mask, -1), tf.shape(token_embeddings)),
            tf.float32
        )

        # Compute the mean-pooled embeddings
        sum_embeddings = tf.math.reduce_sum(token_embeddings * input_mask_expanded, axis=1)
        sum_mask = tf.math.reduce_sum(input_mask_expanded, axis=1)
        return sum_embeddings / tf.clip_by_value(sum_mask, 1e-9, tf.float32.max)
    
    def train_step(self, train_data):
        query = {
            'input_ids': tf.squeeze(train_data['input_ids_query'], axis=1),
            'token_type_ids': tf.squeeze(train_data['token_type_ids_query'], axis=1),
            'attention_mask': tf.squeeze(train_data['attention_mask_query'], axis=1),
        }

        product = {
            'input_ids': tf.squeeze(train_data['input_ids_product'], axis=1),
            'token_type_ids': tf.squeeze(train_data['token_type_ids_product'], axis=1),
            'attention_mask': tf.squeeze(train_data['attention_mask_product'], axis=1),
        }

        labels = train_data['esci_label']

        with tf.GradientTape() as tape:
            # Forward pass for query
            query_predictions = self.model(query)
            pred_query = self.mean_pooling(query_predictions, query['attention_mask'])

            # Forward pass for product
            product_predictions = self.model(product)
            pred_product = self.mean_pooling(product_predictions, product['attention_mask'])

            # Concatenate query and product embeddings
            pred_concat = tf.concat([pred_query, pred_product, tf.abs(pred_query - pred_product)], axis=-1)

            # Compute predictions and loss
            predictions = self.dense(pred_concat)
            loss = self.loss_fn(labels, predictions)

        # Backpropagation
        partial_derivatives = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(partial_derivatives, self.trainable_weights))

        # Update loss metrics
        self.loss_metrics.update_state(loss)

        return {'loss': self.loss_metrics.result()}



    def test_step(self, test_data):
        """
        Implements one test step.
        """
        # Extract query and product inputs
        query = {
            'input_ids': test_data['input_ids_query'],
            'token_type_ids': test_data['token_type_ids_query'],
            'attention_mask': test_data['attention_mask_query'],
        }
        product = {
            'input_ids': test_data['input_ids_product'],
            'token_type_ids': test_data['token_type_ids_product'],
            'attention_mask': test_data['attention_mask_product'],
        }

        labels = test_data['esci_label']

        # Forward pass
        query_predictions = self.model(query)
        pred_query = self.mean_pooling(query_predictions, query['attention_mask'])

        product_predictions = self.model(product)
        pred_product = self.mean_pooling(product_predictions, product['attention_mask'])

        # Concatenate query and product embeddings
        pred_concat = tf.concat([pred_query, pred_product, tf.abs(pred_query - pred_product)], axis=-1)

        # Compute predictions and loss
        predictions = self.dense(pred_concat)
        loss = self.loss_fn(labels, predictions)

        return {'loss': loss}


## Model Training

In [57]:
s_transformer = SentenceTransformer(model)
s_transformer.compile(
    optimizer=Adam(learning_rate=2e-5),
    loss_fn=BinaryCrossentropy()
)

In [58]:
EPOCHS=1
history = s_transformer.fit(tf_dataset, epochs=EPOCHS)

  10/6108 ━━━━━━━━━━━━━━━━━━━━ 8:33:48 5s/step - loss: 0.6939

KeyboardInterrupt: 

In [60]:
model_path = 's_transformer.h5'

In [68]:
# def mean_pooling(model_output, attention_mask):
#     token_embeddings=model_output[0]

#     input_mask_expanded=tf.cast(
#         tf.broadcast_to(tf.expand_dims(attention_mask, -1), tf.shape(token_embeddings)),
#         tf.float32
#     )
#     return tf.math.reduce_sum(token_embeddings*input_mask_expanded, axis=1/tf.clip_by_value(tf.math.reduce_sum(input_mask_expanded, axis=1), 1e-9, tf.float32.max))

In [70]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains token embeddings
    input_mask_expanded = tf.cast(
        tf.broadcast_to(tf.expand_dims(attention_mask, -1), tf.shape(token_embeddings)),
        tf.float32
    )
    # Perform sum and division for mean pooling
    sum_embeddings = tf.math.reduce_sum(token_embeddings * input_mask_expanded, axis=1)
    sum_mask = tf.clip_by_value(tf.math.reduce_sum(input_mask_expanded, axis=1), 1e-9, float('inf'))
    return sum_embeddings / sum_mask


## Testing

### Create embeddings

In [62]:
df_catalogue['product_title'][100000]

'I Melt With You'

In [63]:
product_title = [str(df_catalogue['product_title'][i]) for i in range(len(df_catalogue))]

In [64]:
print(len(product_title))

883868


In [65]:
embeddings = []

In [66]:
INFERENCE_BATCH_SIZE = 640
len(product_title)//INFERENCE_BATCH_SIZE

1381

In [71]:
for i in range(len(product_title)//INFERENCE_BATCH_SIZE):
  tokenized_output=tokenizer(
    product_title[INFERENCE_BATCH_SIZE*i:INFERENCE_BATCH_SIZE*(i+1)],max_length=MAX_LENGTH,padding='max_length',truncation=True,return_tensors="tf")
  model_output=model(tokenized_output)
  embedding=mean_pooling(model_output,tokenized_output['attention_mask'])
  embeddings.append(embedding)
  if i%100==0:
    print(i)

0


KeyboardInterrupt: 

In [ ]:
embeddings

In [ ]:
np.savez_compressed('embeddings.npz', embeddings)
np.savez_compressed('product_titles.npz',product_title)

### using saved embeddings

In [ ]:
loaded_embedding=np.load('embeddings.npz')
loaded_embedding_array=np.array(loaded_embedding['arr_0'])

In [ ]:
loaded_titles=np.load('product_titles.npz')
loaded_titles_array=np.array(loaded_titles['arr_0'])

In [ ]:
loaded_embedding_array=loaded_embedding_array.reshape(-1,loaded_embedding_array.shape[2])
print(loaded_embedding_array.shape)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
inputs = tokenizer(["針なしほっちきす"],max_length=MAX_LENGTH,padding='max_length',truncation=True,return_tensors="tf")

logits = model(**inputs)
out_embedding=mean_pooling(logits,inputs['attention_mask'])
print(out_embedding.shape)

In [ ]:
u_dot_v=np.matmul(loaded_embedding_array,(np.array(out_embedding).T))
print(u_dot_v.shape)

In [ ]:
u_magnitude=np.sqrt(np.sum(loaded_embedding_array*loaded_embedding_array,axis=-1))
print(u_magnitude.shape)
print(u_magnitude)

In [ ]:
v_magnitude=np.sqrt(np.sum(out_embedding*out_embedding,axis=-1))
print(v_magnitude.shape)
print(v_magnitude)

In [ ]:
cosine_similarity=u_dot_v.T/(u_magnitude*v_magnitude)
print(cosine_similarity)

In [ ]:
sorted_indices=np.argsort(cosine_similarity,axis=-1)
print(sorted_indices)

In [ ]:
for i in range(25):
    print(i,loaded_titles_array[sorted_indices[:,len(sorted_indices[0])-i-1]])

In [ ]:
small_dataset = dataset.filter(lambda example: example["query"].startswith("A"))
print(len(small_dataset))